In [1]:
from typing import List
import xmlrpc.client
import socket
from bs4 import BeautifulSoup
import requests
from pathlib import Path


class RPCSender(object):
    def __init__(self, server_url: str, token: str):
        self.server_proxy = xmlrpc.client.ServerProxy(server_url)
        self.token = token
        self.test_connection()

    def test_connection(self):
        try:
            version = self.server_proxy.aria2.getVersion("token:" + self.token)
            print(f"Connected to server. Version: {version['version']}")
        except socket.timeout:
            print("Connection to server timed out.")
        except Exception as e:
            print(f"Failed to connect to server: {e}")

    def download_url(self, url: str):
        options = {"dir": Path(url).parent.parent.parent.name, "out": Path(url).name}
        gid = self.server_proxy.aria2.addUri("token:" + self.token, [url], options)
        print(f"Download started with GID: {gid}")

    def download_urls(self, urls: List[str]):
        for url in urls:
            self.download_url(url)


def get_hub(url):
    url_cut = url.split('https://huggingface.co')[-1].replace('tree', 'resolve')
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    file_links = []

    for link in soup.find_all("a"):
        href = link.get("href")
        if href.startswith(url_cut):
            file_links.append(f"https://huggingface.co{href}")
    for fl in file_links:
        print(fl)
    return file_links

In [2]:
server_url = "http://192.155.1.11:6800/rpc"
token = '113113'
model_url = "https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/tree/main"

rpc = RPCSender(server_url, token)

Connected to server. Version: 1.36.0


In [3]:
download_urls = get_hub(model_url)

https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/.gitattributes
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/README.md
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/added_tokens.json
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/config.json
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/pytorch_model.bin
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/special_tokens_map.json
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/tf_model.h5
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/tokenizer.json
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/tokenizer_config.json
https://huggingface.co/hfl/chinese-electra-180g-small-ex-discriminator/resolve/main/vocab.txt


In [ ]:
rpc.download_urls(urls=download_urls)